In [1]:
import os 
import pandas as pd
import numpy as np
from scipy.io import savemat
from utils.utils import *

In [2]:
import nibabel as nib
threshold = []
for sub in subj_list:
    maskdata_lh_path = os.path.join(mask_dir, sub , f'lh.{sub}.testrois.mgz')
    maskdata_lh = nib.load(maskdata_lh_path).get_fdata().squeeze()
    lh_indices = np.where((maskdata_lh >= 1) & (maskdata_lh <= 15))[0] 
    threshold.append(lh_indices.shape[0])

threshold
    

[17901, 19290, 17885, 13606, 14050, 19115, 13976, 15588]

In [16]:
models_subs = {}
models = ['wself' ,'oself']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'oself'
mode = "train"
rotated = False

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)

models_subs_notrotated = models_subs
#savemat('models_subs_notrotated.mat', models_subs_notrotated)


In [5]:
rotated = True

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)


models_subs_rotated = models_subs
savemat('models_subs_rotated.mat', models_subs_rotated)
models_subs_rotated

{'subj01': {'oself':              x0        y0     sigma      slope  intercept  test_var_explained  \
  0      0.371848 -0.624069  6.235634  30.357092 -30.137138            0.000676   
  1      0.418255 -0.594880  6.208899  25.832947 -25.632463           -0.000803   
  2      0.502787 -0.546511  4.808846  12.609944 -12.428826           -0.002064   
  3      0.349385 -0.618350  5.959802  28.299004 -28.080090           -0.001289   
  4      0.331797 -0.591528  6.096788  30.145597 -29.928112           -0.002998   
  ...         ...       ...       ...        ...        ...                 ...   
  33792  0.141309 -1.050000  5.095603  48.103619 -47.682968            0.010695   
  33793 -0.109746 -1.050000  3.967579  16.174715 -15.943175            0.005295   
  33794 -0.283447  0.284247  4.676534  14.986609 -14.808397            0.008918   
  33795 -0.195607 -0.807038  4.128424  15.912541 -15.684640            0.006280   
  33796 -0.314990 -0.148864  6.484445  29.390841 -29.198298         

In [21]:
rotated = True

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')

        model = np.load(m_file, allow_pickle=True)
        models_subs[sub][m] = {}
        for hemi in hemis:
            
            if hemi == "lh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[:threshold[i]], columns=columns)
            if hemi == "rh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[threshold[i]:], columns=columns)
            models_subs[sub][m][hemi][columns[:-2]] = models_subs[sub][m][hemi][columns[:-2]].astype(np.float32)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('variance_rotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('variance_rotated_rh.csv', index=True)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('test_variance_rotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('test_variance_rotated_rh.csv', index=True)
var_df_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
subj01,0.006189,0.026793,0.029165,0.002278,-0.000325,-0.005879,-0.001703,0.049786,0.002786,0.010252,0.126772,0.039812
subj02,0.077768,0.009943,0.014407,-0.000672,0.011763,-0.004018,-0.000856,0.048991,0.021668,0.015557,0.113827,0.006654
subj03,-0.014228,-0.002869,-0.013792,-0.010635,0.002943,-0.012915,0.010640,0.037972,-0.013382,-0.000838,0.117244,0.011103
subj04,0.031408,0.009487,0.009785,-0.010114,-0.011226,-0.021394,-0.017371,0.032422,0.003609,-0.005136,0.014726,0.012974
subj05,0.009554,0.008693,0.004081,-0.002941,0.005525,0.007579,0.034757,0.070315,-0.003902,0.022845,0.046235,0.037745
subj06,0.015162,0.013580,0.010498,-0.003491,0.000967,0.002377,-0.016791,0.005586,0.004042,0.035878,0.080687,0.009995
subj07,-0.001703,-0.013015,-0.015776,-0.020853,0.003329,-0.015217,-0.008157,-0.006255,-0.068068,-0.010971,0.004940,-0.001562
subj08,-0.005701,-0.005975,-0.000738,-0.000016,-0.003824,-0.004801,0.001324,0.015933,-0.018269,0.009721,0.013792,-0.004892


In [24]:
models_subs = {}
models = ['oself']
columns = ["x0", "y0", "sigma", "slope", "intercept", "test_var_explained", "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'oself'
mode = "train"
hemis = ["lh", "rh"]
rotated = False

for i, sub in enumerate(subj_list):
    models_subs[sub] = {}
    for m in models:
        if rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_rotated.npy')
        if not rotated:
            m_file = os.path.join(models_dir, f'best_fits_{m}_{sub}_{mode}_basevoxel_notrotated.npy')

        model = np.load(m_file, allow_pickle=True)
        models_subs[sub][m] = {}
        for hemi in hemis:
            
            if hemi == "lh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[:threshold[i]], columns=columns)
            if hemi == "rh":
                models_subs[sub][m][hemi] = pd.DataFrame(model[threshold[i]:], columns=columns)
            models_subs[sub][m][hemi][columns[:-2]] = models_subs[sub][m][hemi][columns[:-2]].astype(np.float32)



variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('variance_notrotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('variance_notrotated_rh.csv', index=True)


variance_lh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_lh[i, j] = models_subs[sub][m]["lh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_lh = pd.DataFrame(variance_lh, index=subj_list, columns=rois.keys())
var_df_lh.to_csv('test_variance_notrotated_lh.csv', index=True)

variance_rh = np.zeros((len(subj_list), len(rois.keys())))
m = models[0]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance_rh[i, j] = models_subs[sub][m]["rh"].groupby('roi')['test_var_explained'].mean()[roi]


var_df_rh = pd.DataFrame(variance_rh, index=subj_list, columns=rois.keys())
var_df_rh.to_csv('test_variance_notrotated_rh.csv', index=True)
var_df_rh

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
subj01,0.006322,0.022964,0.028109,0.002443,-0.000174,-0.005756,-0.002013,0.052836,0.002027,0.007845,0.124927,0.034305
subj02,0.077893,0.009897,0.014313,-0.001049,0.011149,-0.004381,-0.001007,0.046422,0.020921,0.012503,0.117033,0.006588
subj03,-0.016984,-0.005185,-0.016858,-0.012669,0.003162,-0.012417,0.009678,0.040301,-0.012257,-0.001416,0.122103,0.012543
subj04,0.030134,0.009651,0.009858,-0.010003,-0.010624,-0.021809,-0.018213,0.025318,0.002238,-0.004077,0.013156,0.012863
subj05,0.009181,0.010704,0.003801,-0.003554,0.005831,0.005436,0.036372,0.058212,-0.002530,0.024504,0.038272,0.047778
subj06,0.015655,0.013313,0.010298,-0.003235,0.000845,0.003681,-0.013372,0.004604,0.003697,0.036066,0.083069,0.009898
subj07,-0.001337,-0.012612,-0.011867,-0.020113,0.002848,-0.015059,-0.007593,-0.006207,-0.065929,-0.009515,0.005660,-0.001597
subj08,-0.004004,-0.006201,-0.001323,0.000234,-0.003931,-0.005005,0.000381,0.012813,-0.021512,0.011862,0.013959,-0.004564


In [ ]:
variance_ = np.zeros((len(subj_list), len(rois.keys())))
m = models[1]
for i, sub in enumerate(subj_list):
    for j, roi in enumerate(rois.keys()):
        variance[i, j] = models_subs[sub][m].groupby('roi')['var_explained'].mean()[roi]